In [7]:
# import all the modules

import os
import requests, zipfile, io
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt


In [4]:
# grab dataset
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/Accidents.zip')

response


<Response [200]>